In [ ]:
"""
Predecir la supervivencia del Titanic

@autor : Raul Vela.
"""

In [23]:
########### LIBRERIAS A UTILIZAR 

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [24]:
########### IMPORTAR LOS DATOS

# Importamos los datos a utilizar desde la web oficial de Kaggle
url_test = 'G:\\Otros ordenadores\\Mi PC\\CURSO BD & IA\\Python_PIA2\\SAA\\2_2_Titanic\\content\\test.csv'
url_train = 'G:\\Otros ordenadores\\Mi PC\CURSO BD & IA\\Python_PIA2\\SAA\\2_2_Titanic\\content\\train.csv'

df_test = pd.read_csv(url_test)
df_train = pd.read_csv(url_train)

#Se guardan los datos en un archivo para siempre tenerlos disponibles
dir_test = 'G:\\Otros ordenadores\\Mi PC\\CURSO BD & IA\\Python_PIA2\\SAA\\2_2_Titanic\\content\\titanic_test.csv'
dir_train = 'G:\\Otros ordenadores\\Mi PC\\CURSO BD & IA\\Python_PIA2\\SAA\\2_2_Titanic\\content\\titanic_train.csv'

df_test.to_csv(dir_test)
df_train.to_csv(dir_train)

#Importar los datos de los archivos .csv almacenados
df_test = pd.read_csv(dir_test)
df_train = pd.read_csv(dir_train)

In [25]:
########### ENTENDIMIENTO DE LOS DATOS

#Verifico la cantidad de datos que hay en los dataset
print('Cantidad de datos:')
print(df_train.shape)
print(df_test.shape)
#Verifico el tipo de datos contenida en ambos dataset
print('Tipos de datos:')
print(df_train.info())
print(df_test.info())
#Verifico los datos faltantes de los dataset
print('Datos faltantes:')
print(pd.isnull(df_train).sum())
print(pd.isnull(df_test).sum())
#Verifico las estadísticas del dataset
print('Estadísticas del dataset:')
print(df_train.describe())
print(df_test.describe())


Cantidad de datos:
(891, 13)
(418, 12)
Tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   891 non-null    int64  
 1   PassengerId  891 non-null    int64  
 2   Survived     891 non-null    int64  
 3   Pclass       891 non-null    int64  
 4   Name         891 non-null    object 
 5   Sex          891 non-null    object 
 6   Age          714 non-null    float64
 7   SibSp        891 non-null    int64  
 8   Parch        891 non-null    int64  
 9   Ticket       891 non-null    object 
 10  Fare         891 non-null    float64
 11  Cabin        204 non-null    object 
 12  Embarked     889 non-null    object 
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dty

In [26]:
##########PREPROCESAMIENTO DE LA DATA 

#Cambio los datos de sexos en números
df_train['Sex'].replace(['female','male'],[0,1],inplace=True)
df_test['Sex'].replace(['female','male'],[0,1],inplace=True)

#Cambio los datos de embarque en números
df_train['Embarked'].replace(['Q','S', 'C'],[0,1,2],inplace=True)
df_test['Embarked'].replace(['Q','S', 'C'],[0,1,2],inplace=True)

#Reemplazo los datos faltantes en la edad por la media de esta columna
print(df_train["Age"].mean())
print(df_test["Age"].mean())
promedio = 30
df_train['Age'] = df_train['Age'].replace(np.nan, promedio)
df_test['Age'] = df_test['Age'].replace(np.nan, promedio)

#Creo varios grupos de acuerdo a bandas de las edades
#Bandas: 0-8, 9-15, 16-18, 19-25, 26-40, 41-60, 61-100
bins = [0, 8, 15, 18, 25, 40, 60, 100]
names = ['1', '2', '3', '4', '5', '6', '7']
df_train['Age'] = pd.cut(df_train['Age'], bins, labels = names)
df_test['Age'] = pd.cut(df_test['Age'], bins, labels = names)

#Se elimina la columna de "Cabin" ya que tiene muchos datos perdidos
df_train.drop(['Cabin'], axis = 1, inplace=True)
df_test.drop(['Cabin'], axis = 1, inplace=True)

#Elimino las columnas que considero que no son necesarias para el analisis
df_train = df_train.drop(['PassengerId','Name','Ticket'], axis=1)
df_test = df_test.drop(['Name','Ticket'], axis=1)

#Se elimina las filas con los datos perdidos
df_train.dropna(axis=0, how='any', inplace=True)
df_test.dropna(axis=0, how='any', inplace=True)

#Verifico los datos
print(pd.isnull(df_train).sum())
print(pd.isnull(df_test).sum())

print(df_train.shape)
print(df_test.shape)

print(df_test.head())
print(df_train.head())

29.69911764705882
30.272590361445783
Unnamed: 0    0
Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64
Unnamed: 0     0
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64
(889, 9)
(417, 9)
   Unnamed: 0  PassengerId  Pclass  Sex Age  SibSp  Parch     Fare  Embarked
0           0          892       3    1   5      0      0   7.8292         0
1           1          893       3    0   6      1      0   7.0000         1
2           2          894       2    1   7      0      0   9.6875         0
3           3          895       3    1   5      0      0   8.6625         1
4           4          896       3    0   4      1      1  12.2875         1
   Unnamed: 0  Survived  Pclass  Sex Age  SibSp  Parch     Fare  Embarked
0           0         0       3    1   4      1      0   7.2500       1.0
1           

In [28]:
##########APLICACIÓN DE ALGORITMOS DE MACHINE LEARNING 

#Separo la columna con la información de los sobrevivientes
X = np.array(df_train.drop(['Survived'], axis=1))
y = np.array(df_train['Survived'])

#Separo los datos de "train" en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

##Regresión logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
print('Precisión Regresión Logística:')
print(logreg.score(X_train, y_train))

##Support Vector Machines
svc = SVC()
svc.fit(X_train, y_train)
Y_pred = svc.predict(X_test)
print('Precisión Soporte de Vectores:')
print(svc.score(X_train, y_train))

##KN neighbors
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
Y_pred = knn.predict(X_test)
print('Precisión Vecinos más Cercanos:')
print(knn.score(X_train, y_train))

Precisión Regresión Logística:
0.7651195499296765
Precisión Soporte de Vectores:
0.6568213783403657
Precisión Vecinos más Cercanos:
0.8171589310829818


c:\Users\Robot\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
##########PREDICCIÓN UTILIZANDO LOS MODELOS

ids = df_test['PassengerId']

##Regresión logística
prediccion_logreg = logreg.predict(df_test.drop('PassengerId', axis=1).values)
out_logreg = pd.DataFrame({ 'PassengerId' : ids, 'Survived': prediccion_logreg })
print('Predicción Regresión Logística:')
print(out_logreg.head())


##Support Vector Machines
prediccion_svc = svc.predict(df_test.drop('PassengerId', axis=1))
out_svc = pd.DataFrame({ 'PassengerId' : ids, 'Survived': prediccion_svc })
print('Predicción Soporte de Vectores:')
print(out_svc.head())

##K neighbors
prediccion_knn = knn.predict(df_test.drop('PassengerId', axis=1))
out_knn = pd.DataFrame({ 'PassengerId' : ids, 'Survived': prediccion_knn })
print('Predicción Vecinos más Cercanos:')
print(out_knn.head())

Predicción Regresión Logística:
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         0
Predicción Soporte de Vectores:
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         0
Predicción Vecinos más Cercanos:
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         0


c:\Users\Robot\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
c:\Users\Robot\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
